<h2>Ben Gruher Lab 2</h2>

In [2]:
sc

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<h4>Helper functions and global variables</h4>

In [3]:
# S3 path locations for parquet files

OBSERVATION_OUTPUT_LOCATION = 's3://<BUCKET_NAME>/gruherb/observations/'
AIRCRAFT_OUTPUT_LOCATION = 's3://<BUCKET_NAME>/gruherb/aircraft/'
OPERATOR_OUTPUT_LOCATION = 's3://<BUCKET_NAME>/gruherb/operator/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# list of field names (includes Reg for filtering US flights)

OBSERVATION_FIELDS = ['Icao', 'Reg', 'Alt', 'Lat', 'Long', 'PosTime', 'Spd', 'From', 'To', 'Gnd']
AIRCRAFT_FIELDS = ['Icao', 'Reg', 'Species', 'Mil', 'Cou', 'Type']
OPERATOR_FIELDS = ['Icao', 'Reg', 'Op', 'OpIcao']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# helper function - converts files into JSON objects

import json

def jsonify(s):
    try:
        s = s.replace("\n", "")
        j = json.loads(s)
        return j['acList']
    except:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# helper function - takes fields relavant for observations

import datetime

def procPosTime(posTime):
    return datetime.datetime.fromtimestamp(int(posTime)/1000.0)

def extractAndTransformObservations(j):
    try:
        l = list(map(lambda f: j.get(f, None), OBSERVATION_FIELDS))
        i = OBSERVATION_FIELDS.index('PosTime')
        if l[i] != None:
            l[i] = procPosTime(l[i])
        return l
    except:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# helper function - takes fields relavant for aircraft

def extractAndTransformAircraft(j):
    try:
        l = list(map(lambda f: j.get(f, None), AIRCRAFT_FIELDS))
        return l
    except:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# helper function - takes fields relavant for operator

def extractAndTransformOperator(j):
    try:
        l = list(map(lambda f: j.get(f, None), OPERATOR_FIELDS))
        return l
    except:
        return None

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql.types import StringType, IntegerType, LongType, DoubleType, BooleanType, TimestampType
from pyspark.sql.types import StructType, StructField

observation_schema = StructType(
    [
        StructField('Icao', StringType(), False),
        StructField('Alt', IntegerType(), True),
        StructField('Lat', DoubleType(), True),
        StructField('Long', DoubleType(), True),
        StructField('PosTime', TimestampType(), True),
        StructField('Spd', DoubleType(), True),
        StructField('From', StringType(), True),
        StructField('To', StringType(), True),
        StructField('Gnd', BooleanType(), True)
    ]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
aircraft_schema = StructType(
    [
        StructField('Icao', StringType(), False),
        StructField('Species', IntegerType(), True),
        StructField('Mil', BooleanType(), True),
        StructField('Cou', StringType(), True),
        StructField('Type', StringType(), True)
    ]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
operator_schema = StructType(
    [
        StructField('Icao', StringType(), False),
        StructField('Op', StringType(), True),
        StructField('OpIcao', StringType(), True)
    ]
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h4>Main function:</h4>

In [12]:
def process_days(list_of_days):
    for day in list_of_days:
        input_path = f"s3://<BUCKET_NAME>/{day}/*.bz2"
        
        # the line below creates an RDD that takes only the file contents, jsonifies the files, filters the Nones
        observations = sc.wholeTextFiles(input_path, minPartitions=3600).map(lambda t: t[1]).flatMap(jsonify).filter(lambda d: d is not None)
        
        # branch for observations
        obs_0 = observations.map(extractAndTransformObservations).filter(lambda l: l is not None)
        obs_1 = obs_0.filter(lambda l: l[1] != None and len(l[1]) > 0 and l[1].startswith('N'))
        obs_2 = obs_1.map(lambda l: [l[0]] + l[2:]).filter(lambda l: l[0] is not None)
        obs_2.take(2)
        obs_df = spark.createDataFrame(obs_2, observation_schema)
        obs_df.write.parquet(OBSERVATION_OUTPUT_LOCATION + day)
        
        # branch for aircraft
        air_0 = observations.map(extractAndTransformAircraft).filter(lambda l: l is not None)
        air_1 = air_0.filter(lambda l: l[1] != None and len(l[1]) > 0 and l[1].startswith('N'))
        air_2 = air_1.map(lambda l: [l[0]] + l[2:]).filter(lambda l: l[0] is not None)
        air_2.take(2)
        air_df = spark.createDataFrame(air_2, aircraft_schema).dropDuplicates()
        air_df.write.parquet(AIRCRAFT_OUTPUT_LOCATION + day)
        
        # branch for operator
        op_0 = observations.map(extractAndTransformOperator).filter(lambda l: l is not None)
        op_1 = op_0.filter(lambda l: l[1] != None and len(l[1]) > 0 and l[1].startswith('N'))
        op_2 = op_1.map(lambda l: [l[0]] + l[2:]).filter(lambda l: l[0] is not None)
        op_2.take(2)
        op_df = spark.createDataFrame(op_2, operator_schema).dropDuplicates()
        op_df.write.parquet(OPERATOR_OUTPUT_LOCATION + day)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h4>Analysis</h4>

In [13]:
daysOfInterest = ['2020-10-15', '2020-10-16']

operatorDataPathDayOne = 's3://<BUCKET_NAME>/gruherb/operator/2020-10-15'
operatorDataPathDayTwo = 's3://<BUCKET_NAME>/gruherb/operator/2020-10-16'

observationDataPathDayOne = 's3://<BUCKET_NAME>/gruherb/observations/2020-10-15'
observationDataPathDayTwo = 's3://<BUCKET_NAME>/gruherb/observations/2020-10-16'

aircraftDataPathDayOne = 's3://<BUCKET_NAME>/gruherb/aircraft/2020-10-15'
aircraftDataPathDayTwo = 's3://<BUCKET_NAME>/gruherb/aircraft/2020-10-16'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# commented so this cell does not get executed again if 'Run all cells' is selected (this process takes about an hour)
# process_days(daysOfInterest)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
observationDayOneDf = spark.read.parquet(observationDataPathDayOne)
observationDayTwoDf = spark.read.parquet(observationDataPathDayTwo)
observationDf = observationDayOneDf.union(observationDayTwoDf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
aircraftDayOneDf = spark.read.parquet(aircraftDataPathDayOne)
aircraftDayTwoDf = spark.read.parquet(aircraftDataPathDayTwo)
aircraftDf = aircraftDayOneDf.union(aircraftDayTwoDf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
operatorDayOneDf = spark.read.parquet(operatorDataPathDayOne)
operatorDayTwoDf = spark.read.parquet(operatorDataPathDayTwo)
operatorDf = operatorDayOneDf.union(operatorDayTwoDf)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<h5>1. Number of observations by aircraft</h5>

In [18]:
aircraftObsDf = observationDf.join(aircraftDf, on=['Icao'], how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# 20 aircraft with the most observations
aircraftObsDf.groupBy('Icao').count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+
|  Icao|count|
+------+-----+
|A11062|22150|
|A16927|22150|
|A1FFB5|22022|
|A1F0D9|20606|
|A26F6D|18982|
|A0B8FB|18794|
|ACCA19|18308|
|A3965C|17696|
|A7DA54|17682|
|ABB6B5|17528|
|A3650B|17418|
|A37A02|17406|
|A88CCF|16802|
|A67883|16568|
|A033FF|16560|
|ADDA24|16470|
|A89086|16146|
|A0DDFB|16102|
|AC10D8|15958|
|A10BBC|15864|
+------+-----+
only showing top 20 rows

In [20]:
# number of aircraft with less than 240 observations in 2 days
obsPerAircraft = aircraftObsDf.groupBy('Icao').count()
obsPerAircraft.filter(obsPerAircraft['count'] < 240).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

16293

In [21]:
# total number of aircraft
aircraftDf.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

61838

In [38]:
sc.install_pypi_package('matplotlib')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: numpy
    Found existing installation: numpy 1.14.5
    Not uninstalling numpy at /usr/local/lib64/python3.6/site-packages, outside environment /tmp/1611190330513-0
    Can't uninstall 'numpy'. No files were found to uninstall.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
python36-sagemaker-pyspark 1.2.6 requires pyspark==2.3.2, which is not installed.

In [55]:
n = 10
bins, counts = aircraftObsDf.groupBy('Icao').count().select("count").rdd.flatMap(lambda x: x).histogram(n)

print('Bins: ', bins)
print('Counts: ', counts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Bins:  [1.0, 2215.9, 4430.8, 6645.700000000001, 8860.6, 11075.5, 13290.400000000001, 15505.300000000001, 17720.2, 19935.100000000002, 22150]
Counts:  [35075, 4150, 2265, 1505, 461, 169, 60, 17, 3, 4]

In [56]:
import matplotlib.pyplot as plt
plt.hist(bins[:-1], bins=bins, weights=counts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Matplotlib requires numpy>=1.15; you have 1.14.5
Traceback (most recent call last):
  File "/tmp/1611190330513-0/local/lib64/python3.6/site-packages/matplotlib/__init__.py", line 174, in <module>
    _check_versions()
  File "/tmp/1611190330513-0/local/lib64/python3.6/site-packages/matplotlib/__init__.py", line 171, in _check_versions
    .format(modname, minver, module.__version__))
ImportError: Matplotlib requires numpy>=1.15; you have 1.14.5



<h5>2. From and To fields</h5>

In [23]:
# join observation and operator
opObJoined = observationDf.join(operatorDf, on=['Icao'], how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
keyOperators = ['Alaska Airlines', 'Southwest Airlines', 'JetBlue', 'American Airlines', 'United Airlines']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
from pyspark.sql.functions import col
mainAirlines = opObJoined.filter(col('Op').isin(keyOperators))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# number of observations for each airline
mainAirlines.groupBy('Op').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------+
|                Op|  count|
+------------------+-------+
|   United Airlines|2331135|
| American Airlines|4322836|
|           JetBlue|   6938|
|   Alaska Airlines|1193859|
|Southwest Airlines|3432734|
+------------------+-------+

In [27]:
# fill rate for From and To fields
totalObs = mainAirlines.count()
fromFill = mainAirlines.filter(col('From') != 'null').count() / totalObs
toFill = mainAirlines.filter(col('To') != 'null').count() / totalObs
print('From field fill rate: ', fromFill)
print('To field fill rate: ', toFill)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

From field fill rate:  0.8770013949942157
To field fill rate:  0.8770013949942157

In [36]:
# consistency check for aircraft observations "close in time" (we will use a time interval of +- 10 minutes)
from pyspark.sql import functions as F
import pyspark.sql.functions

interval = 60 * 10    # 10 minutes
timeDf = mainAirlines.withColumn(
    'time_interval',
    pyspark.sql.functions.from_unixtime(pyspark.sql.functions.floor(pyspark.sql.functions.unix_timestamp(mainAirlines['PosTime']) / interval) * interval)
)

print('From column:')
uniqueFroms = timeDf.groupBy(['Icao', 'time_interval']).agg(F.collect_set('From')).withColumn('UniqueFrom', F.size('collect_set(From)')).orderBy('UniqueFrom', ascending=False).filter(col('time_interval') != 'null')
uniqueFroms.show()
multipleFroms = uniqueFroms.filter(col('UniqueFrom') > 1).count() / uniqueFroms.count()
print('Percentage of similarily-timed observations from the same plane with multiple \'From\' values: ', multipleFroms)

print('To column:')
uniqueTos = timeDf.groupBy(['Icao', 'time_interval']).agg(F.collect_set('To')).withColumn('UniqueTo', F.size('collect_set(To)')).orderBy('UniqueTo', ascending=False).filter(col('time_interval') != 'null')
uniqueTos.show()
multipleTos = uniqueTos.filter(col('UniqueTo') > 1).count() / uniqueTos.count()
print('Percentage of similarily-timed observations from the same plane with multiple \'To\' values: ', multipleTos)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

From column:
+------+-------------------+--------------------+----------+
|  Icao|      time_interval|   collect_set(From)|UniqueFrom|
+------+-------------------+--------------------+----------+
|AA0E62|2020-10-16 22:10:00|[PHX Phoenix Sky ...|         2|
|A4ACAE|2020-10-16 16:40:00|[DFW Dallas Fort ...|         2|
|AAE582|2020-10-16 23:00:00|[CLT Charlotte Do...|         2|
|A4F59C|2020-10-15 14:30:00|[DFW Dallas Fort ...|         2|
|AB4EA8|2020-10-16 15:30:00|[PHL Philadelphia...|         2|
|AA64CA|2020-10-15 00:00:00|[DFW Dallas Fort ...|         2|
|AA7DAD|2020-10-15 15:50:00|[RDU Raleigh Durh...|         2|
|AC90A7|2020-10-16 23:00:00|[DFW Dallas Fort ...|         2|
|AD9BC8|2020-10-16 21:40:00|[CLT Charlotte Do...|         2|
|AA1BE0|2020-10-16 22:40:00|[DFW Dallas Fort ...|         2|
|ADC8E7|2020-10-15 14:50:00|[DFW Dallas Fort ...|         2|
|AC7216|2020-10-15 16:40:00|[FAT Fresno Yosem...|         2|
|A0B199|2020-10-15 14:00:00|[MDT Harrisburg, ...|         2|
|A9034C|202

<h5>3. Aircraft attributes</h5>

In [29]:
db_lookup_attributes = ['Species', 'Mil', 'Cou', 'Type']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# fill rate calculation is different for each data type
numObs = aircraftObsDf.count()

print('Species fill rate: ') 
print(aircraftObsDf.filter(col('Species') != 0).count() / numObs)

print('Mil fill rate: ')
nonNullMil = aircraftObsDf.filter(col('Mil') == True).count() + aircraftObsDf.filter(col('Mil') == False).count()
print(nonNullMil / numObs)

print('Cou fill rate: ')
print(aircraftObsDf.filter(col('Cou') != 'null').count() / numObs)

print('Type fill rate: ')
print(aircraftObsDf.filter(col('Type') != 'null').count() / numObs)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Species fill rate: 
0.9925651663329473
Mil fill rate: 
1.0
Cou fill rate: 
1.0
Type fill rate: 
0.996097283534554

In [31]:
from pyspark.sql import functions as F
from pyspark.sql.functions import size

# for each attribute, report number of unique values for each plane
for attribute in db_lookup_attributes:
    print(attribute, 'unique values per flight:')
    
    colName = 'Unique{}'.format(attribute)
    generatedColName = 'collect_set({})'.format(attribute)
    uniqueValues = aircraftDf.groupBy('Icao').agg(F.collect_set(attribute)).withColumn(colName, F.size(generatedColName)).orderBy(colName, ascending=False)
    
    # special case - species, need to filter out values of 0
    if attribute == 'Species':
        uniqueValues = aircraftDf.filter(col(attribute) != 0).groupBy('Icao').agg(F.collect_set(attribute)).withColumn(colName, F.size(generatedColName)).orderBy(colName, ascending=False)
    uniqueValues.show()
    multipleValues = uniqueValues.filter(col(colName) > 1).count() / uniqueValues.count()
    print('Percentage of planes with multiple {} values: {}'.format(attribute, multipleValues))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Species unique values per flight:
+------+--------------------+-------------+
|  Icao|collect_set(Species)|UniqueSpecies|
+------+--------------------+-------------+
|A003F0|                 [1]|            1|
|A01C13|                 [1]|            1|
|A01E18|                 [1]|            1|
|A0267E|                 [4]|            1|
|A027AF|                 [1]|            1|
|A0522B|                 [1]|            1|
|A05283|                 [1]|            1|
|A05514|                 [1]|            1|
|A05553|                 [1]|            1|
|A05DC8|                 [1]|            1|
|A05FCD|                 [1]|            1|
|A06048|                 [1]|            1|
|A064C8|                 [1]|            1|
|A066AA|                 [1]|            1|
|A07485|                 [1]|            1|
|A07898|                 [1]|            1|
|A0808D|                 [1]|            1|
|A08812|                 [1]|            1|
|A08EEC|                 [4]|            1

<h5>4. Aircraft to operator</h5>

In [32]:
db_lookup_attributes2 = ['Op', 'OpIcao']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# join aircraft and operator on ICAO
airOpJoined = aircraftDf.join(operatorDf, on=['Icao'], how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
airOpJoined.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+-----+-------------+----+--------------------+------+
|  Icao|Species|  Mil|          Cou|Type|                  Op|OpIcao|
+------+-------+-----+-------------+----+--------------------+------+
|A2D382|      1|false|United States|MD11|United Parcel Ser...|   UPS|
|A2D382|      1|false|United States|MD11|United Parcel Ser...|   UPS|
|A3E584|      1|false|United States|CL35|           TWAPA Inc|  null|
|A03C7E|      1|false|United States|GL5T|             NetJets|   EJA|
|A03C7E|      1|false|United States|GL5T|             NetJets|   EJA|
|A79CF4|      1|false|United States|C172|             Private|  null|
|A79CF4|      1|false|United States|C172|             Private|  null|
|ABADC0|      1|false|United States|B738|  Southwest Airlines|   SWA|
|ABADC0|      1|false|United States|B738|  Southwest Airlines|   SWA|
|A6B7EC|      1|false|United States|B738|     Alaska Airlines|   ASA|
|A6B7EC|      1|false|United States|B738|     Alaska Airlines|   ASA|
|A08F6A|      1|fals

In [35]:
for attribute in db_lookup_attributes2:
    print('Attribute: ', attribute)
    fillRate = airOpJoined.filter(col(attribute) != 'null').count() / airOpJoined.count()
    print('Fill rate: ', fillRate)
    
    colName = 'Unique{}'.format(attribute)
    generatedColName = 'collect_set({})'.format(attribute)
    uniqueValues = airOpJoined.groupBy('Icao').agg(F.collect_set(attribute)).withColumn(colName, F.size(generatedColName)).orderBy(colName, ascending=False)
    uniqueValues.show()
    
    multipleValues = uniqueValues.filter(col(colName) > 1).count() / uniqueValues.count()
    print('Percentage of planes with multiple {} values: {}'.format(attribute, multipleValues))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Attribute:  Op
Fill rate:  0.9999482112153792
+------+--------------------+--------+
|  Icao|     collect_set(Op)|UniqueOp|
+------+--------------------+--------+
|A0026E|[AUBURN UNIVERSIT...|       1|
|A01EE6|[San Diego County...|       1|
|A02FFF|[MCDANIEL TRENTON...|       1|
|A03F75|           [Private]|       1|
|A03F84| [Republic Airlines]|       1|
|A0492E|[PERFORMANCE BY D...|       1|
|A05D05|[KENNEDY VIRGIL C...|       1|
|A05EDF|[Airbus Helicopte...|       1|
|A062CC|[N124CM LLC     -...|       1|
|A07ECC|[BRODIE JAMES W  ...|       1|
|A095B4|             [FedEx]|       1|
|A09977|[IRF ASSOCIATES L...|       1|
|A0A21A|[MILAM EQUIPMENT ...|       1|
|A0C963|[CLINKSCALES RADI...|       1|
|A0CCAB|[SCHAUS RICHARD M...|       1|
|A0CF0E|        [Ken Wagner]|       1|
|A0EFDF|[L-3 Communicatio...|       1|
|A0FCD6|[WELLS FARGO BANK...|       1|
|A101C3| [American Airlines]|       1|
|A11AD4|  [SkyWest Airlines]|       1|
+------+--------------------+--------+
only showing top 2